In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.random_projection import GaussianRandomProjection
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn import metrics
import shap
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
from utils import *
from skgarden import RandomForestQuantileRegressor

%matplotlib notebook
%pylab

D:\Programs\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
D:\Programs\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=n

Using matplotlib backend: nbAgg
Populating the interactive namespace from numpy and matplotlib


`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).


In [13]:
def ious(path, path2, p):
    data = pd.read_csv(path+'/data.csv')
    _, _, X, _, Y = data_to_input(data)
    rf = RandomForestQuantileRegressor(
    random_state=0, n_estimators=1000)
    rf.fit(X,Y)
    
    ious = []
    for i in tqdm(range(1,301)):
        sample = pd.read_csv(path2 + "/sample"+str(i)+".csv")
        Xt, Yt = data_to_quality(sample)
        if Xt.shape[0] > 0:
            ious.append(iou(rf.predict(Xt, (100-p)/2), rf.predict(Xt, 100-(100-p)/2), Yt, p))
    
    return np.array(ious)

In [4]:
unos = ious('./Rate=UNOS/Batch Simulation Sun Jan 10 23-35-55 EST 2021', './Rate=UNOS', 95)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [01:35<00:00,  3.13it/s]


In [6]:
rate3 = ious('./Rate=3/Batch Simulation Tue May 18 22-39-09 EDT 2021', './Rate=3', 95)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [01:35<00:00,  3.14it/s]


In [7]:
rate2 = ious('./Rate=2/Batch Simulation Sun Mar 14 01-58-15 EST 2021', './Rate=2', 95)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [01:27<00:00,  3.41it/s]


In [8]:
rate1 = ious('./Rate=1/Batch Simulation Mon May 17 00-21-59 EDT 2021', './Rate=1', 95)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [01:26<00:00,  3.46it/s]


In [25]:
rate1l = ious('./Rate=1, Long/Long', './Rate=1, Long', 95)

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [01:39<00:00,  3.02it/s]


In [14]:
rate2l = ious('./Rate=2, Long/Long', './Rate=2, Long', 95)

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [01:34<00:00,  3.19it/s]


In [19]:
print(unos.mean())

0.8118044006472603


In [20]:
print(rate3.mean())

0.7976524945011935


In [21]:
print(rate2.mean())

0.8153687933096516


In [22]:
print(rate1.mean())

0.7473050571919283


In [26]:
print(rate1l.mean())

0.6323888302097196


In [15]:
print(rate2l.mean())

0.8085634016796548


In [5]:
print(np.percentile(unos,50))

0.815379027027645


In [9]:
print(np.percentile(rate3,50))

0.7872536404298458


In [10]:
print(np.percentile(rate2,50))

0.8326805142667728


In [11]:
print(np.percentile(rate1,50))

0.7592913422264613


In [2]:
def fedious(q,p):
    data = pd.read_csv('./Federated'+str(q)+'/1/data.csv')
    _, _, X, _, Y = data_to_input(data)
    for i in range(2,6):
        datai = pd.read_csv('./Federated'+str(q)+'/'+str(i)+'/data.csv')
        _, _, Xi, _, Yi = data_to_input(datai)
        X = pd.concat([X,Xi])
        Y = pd.concat([Y,Yi])
    rf = RandomForestQuantileRegressor(
    random_state=1, min_samples_split=10, n_estimators=1000)
    rf.fit(X,Y)
    
    ious = []
    for i in tqdm(range(1,301)):
        sample = pd.read_csv("./Federated"+str(q)+"/sample"+str(i)+".csv")
        Xt, Yt = data_to_quality(sample)
        if Xt.shape[0] > 0:
            ious.append(iou(rf.predict(Xt, (100-p)/2), rf.predict(Xt, 100-(100-p)/2), Yt, p))
    
    return np.array(ious)

In [3]:
fedrateunos=fedious('UNOS',95)

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [01:32<00:00,  3.26it/s]


In [4]:
fedrate3=fedious(3,95)

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [01:31<00:00,  3.28it/s]


In [5]:
fedrate2 = fedious(2,95)

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [01:33<00:00,  3.22it/s]


In [6]:
fedrate1=fedious(1,95)

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [01:28<00:00,  3.38it/s]


In [7]:
fedrateunos.mean()

0.8164816373176303

In [8]:
fedrate3.mean()

0.815856349113708

In [9]:
fedrate2.mean()

0.8148090216806568

In [10]:
fedrate1.mean()

0.8115767333563735

In [16]:
fedrate2l = fedious('2l',95)

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [01:39<00:00,  3.02it/s]


In [17]:
fedrate2l.mean()

0.819755036829994

In [18]:
rate1l2 = ious('./Rate=1, Long2/Long', './Rate=1, Long2', 95)

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [01:33<00:00,  3.22it/s]


In [19]:
rate1l2.mean()

0.7613805077825485

In [20]:
fedrate1l = fedious('1l',95)

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [01:34<00:00,  3.18it/s]


In [21]:
fedrate1l.mean()

0.8000935211041894